# Re-creating Weinberg et al. 2017 one-zone model results in VICE

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, ScalarFormatter, NullFormatter
import vice
from vice.yields.sneia import iwamoto99

# VICE yields settings
vice.yields.ccsne.settings['o'] = 0.015
vice.yields.ccsne.settings['fe'] = 0.0012

# Plot settings
plt.rc(('xtick', 'ytick'), direction='in')
plt.rc(('xtick.minor', 'ytick.minor'), size=4)
plt.rc(('xtick.major', 'ytick.major'), size=8)
plt.rc('xtick', top=True)
plt.rc('ytick', right=True)
plt.rc('font', family='STIXgeneral')
plt.rc('figure', dpi=140)
plt.rc('figure', figsize=(4, 4))

ImportError: Error importing VICE. If you conducted this installation with pip, it is likely there is not a binary installer for this operating system and version of python. 
To solve this, use pip to uninstall VICE, and then install from source. If you have installed from source, see the section of the documentation entitled "Troubleshooting Your Build" at the following link: 
https://vice-astro.readthedocs.io/en/latest/install.html#troubleshooting-your-build


In [ ]:
# Set up multiple models
names = ['W17_fid', 'W17_long_sfh', 'W17_short_sfh', 'W17_linexp', 
         'W17_short_ia', 'W17_long_ia', 'W17_short_delay', 'W17_long_delay',
         'W17_high_outflow', 'W17_low_outflow', 'W17_long_sfe']
tau_sfh = [6, 40, 2.5, 2.5, 6, 6, 6, 6] # star formation timescale [Gyr]
tau_ia = [1.5, 1.5, 1.5, 1.5, 0.5, 4.5, 1.5, 1.5]
delay = [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.05, 0.45]
eta = [2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 5., 1., 2.5]
tau_star = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3]

for i, name in enumerate(names):
    sz = vice.singlezone(name=name)

    def exp_sfr(t):
        """Exponentially declining star formation rate."""
        sfr = np.exp(-t/tau_sfh[i])
        if name == 'W17_linexp':
            sfr *= t
        return sfr

    sz.func = exp_sfr
    sz.mode = 'sfr'
    sz.elements = ('fe', 'o')
    sz.IMF = 'kroupa'
    sz.eta = eta[i] # mass-loading parameter, or ratio of outflow rate to star formation rate
    sz.recycling = 0.4 # recycling parameter, fraction of stellar mass from CCSNe and AGB stars returned at original metallicity
    sz.delay = delay[i] # delay time of onset of SNe Ia [Gyr]
    sz.RIa = 'exp' # SN Ia DTD
    sz.tau_ia = tau_ia[i] # exponential SN Ia timescale [Gyr]
    sz.tau_star = tau_star[i] # SFE timescale, ratio of current gas mass to star formation rate [Gyr]
    
    sz.run(np.linspace(0, 12.5, 1251), overwrite=True)

In [ ]:
history = vice.history('W17_fid')
mdf = vice.mdf('W17_fid')
print(history)
print(mdf)

In [ ]:
# Abundance evolution and O-Fe tracks, adjusting star formation history
z_fe_sun = 0.0012
z_o_sun = 0.0056

fig, axs = plt.subplots(1, 3, figsize=(12, 4))

# Plot other tracks
names = ['W17_fid', 'W17_long_sfh', 'W17_short_sfh', 'W17_linexp']
colors = ['k', 'r', 'g', 'b']
labels = ['$\\tau_{sfh}=6.0$ Exp', '$\\tau_{sfh}=40.0$ Exp', '$\\tau_{sfh}=2.5$ Exp', '$\\tau_{sfh}=2.5$ Lin-Exp']
for i, name in enumerate(names):
    history = vice.history(name)
    axs[0].plot(history['time'][1:], np.log10(np.array(history['z(fe)'][1:])/z_fe_sun), c=colors[i], label='Fe')
    axs[0].plot(history['time'][1:], np.log10(np.array(history['z(o)'][1:])/z_o_sun), c=colors[i], linestyle=':', label='O')
    axs[1].plot(history['time'][1:], np.log10(np.array(history['z(fe)'][1:])/z_fe_sun), c=colors[i], label='Fe')
    axs[1].plot(history['time'][1:], np.log10(np.array(history['z(o)'][1:])/z_o_sun), c=colors[i], linestyle=':', label='O')
    axs[2].plot(history['[fe/h]'][1:], history['[o/fe]'][1:], c=colors[i], label=labels[i])

# Format plot
ax = axs[0]
ax.set_xlim((-1, 13.5))
ax.set_ylim((-1.5, 0.6))
ax.xaxis.set_minor_locator(MultipleLocator(1))
ax.xaxis.set_major_locator(MultipleLocator(5))
ax.yaxis.set_minor_locator(MultipleLocator(0.1))
ax.yaxis.set_major_locator(MultipleLocator(0.5))
ax.set_xlabel('t [Gyr]')
ax.set_ylabel('log Z/Z$_\odot$')
ax.legend(['Fe', 'O'], loc='upper left')

ax = axs[1]
ax.set_xscale('log')
ax.set_xlim((0.05, 20))
ax.set_ylim((-1.5, 0.6))
ax.xaxis.set_major_formatter(ScalarFormatter())
ax.yaxis.set_minor_locator(MultipleLocator(0.1))
ax.yaxis.set_major_locator(MultipleLocator(0.5))
ax.set_xlabel('t [Gyr]')
ax.set_ylabel('log Z/Z$_\odot$')
ax.legend(['Fe', 'O'], loc='upper left')

ax = axs[2]
ax.set_xlim((-2, 0.5))
ax.set_ylim((-0.3, 0.5))
ax.xaxis.set_minor_locator(MultipleLocator(0.2))
ax.xaxis.set_major_locator(MultipleLocator(1))
ax.yaxis.set_minor_locator(MultipleLocator(0.05))
ax.yaxis.set_major_locator(MultipleLocator(0.2))
ax.set_xlabel('[Fe/H]')
ax.set_ylabel('[O/Fe]')
ax.legend(loc='lower left')

plt.show()

In [ ]:
# Abundance evolution and O-Fe tracks, adjusting SN Ia DTD
z_fe_sun = 0.0012
z_o_sun = 0.0056

fig, axs = plt.subplots(1, 3, figsize=(12, 4))

# Plot other tracks
names = ['W17_fid', 'W17_short_ia', 'W17_long_ia', 'W17_short_delay', 'W17_long_delay']
colors = ['k', 'r', 'g', 'cyan', 'b']
labels = ['$\\tau_{Ia}=1.5, t_d=0.15$', '$\\tau_{Ia}=0.5, t_d=0.15$', '$\\tau_{Ia}=5.5, t_d=0.15$', '$\\tau_{Ia}=1.5, t_d=0.05$', '$\\tau_{Ia}=1.5, t_d=0.45$']
for i, name in enumerate(names):
    history = vice.history(name)
    axs[0].plot(history['time'][1:], np.log10(np.array(history['z(fe)'][1:])/z_fe_sun), c=colors[i])
    axs[1].plot(history['time'][1:], np.log10(np.array(history['z(fe)'][1:])/z_fe_sun), c=colors[i])
    axs[2].plot(history['[fe/h]'][1:], history['[o/fe]'][1:], c=colors[i], label=labels[i])

# Format plot
ax = axs[0]
ccsne_yields = vice.output('W17_fid').ccsne_yields
print(ccsne_yields)
ax.scatter(history['time'][-1], np.log10(ccsne_yields['fe']/z_fe_sun), marker='o', facecolors='none', edgecolors='k', label='CCSN')
ax.set_xlim((-1, 13.5))
ax.set_ylim((-1.5, 0.6))
ax.xaxis.set_minor_locator(MultipleLocator(1))
ax.xaxis.set_major_locator(MultipleLocator(5))
ax.yaxis.set_minor_locator(MultipleLocator(0.1))
ax.yaxis.set_major_locator(MultipleLocator(0.5))
ax.set_xlabel('t [Gyr]')
ax.set_ylabel('log Z/Z$_\odot$')
ax.legend(loc='upper left')

ax = axs[1]
ax.set_xscale('log')
ax.set_xlim((0.05, 20))
ax.set_ylim((-1.5, 0.6))
ax.xaxis.set_major_formatter(ScalarFormatter())
ax.yaxis.set_minor_locator(MultipleLocator(0.1))
ax.yaxis.set_major_locator(MultipleLocator(0.5))
ax.set_xlabel('t [Gyr]')
ax.set_ylabel('log Z/Z$_\odot$')

ax = axs[2]
ax.set_xlim((-2, 0.5))
ax.set_ylim((-0.3, 0.5))
ax.xaxis.set_minor_locator(MultipleLocator(0.2))
ax.xaxis.set_major_locator(MultipleLocator(1))
ax.yaxis.set_minor_locator(MultipleLocator(0.05))
ax.yaxis.set_major_locator(MultipleLocator(0.2))
ax.set_xlabel('[Fe/H]')
ax.set_ylabel('[O/Fe]')
ax.legend(loc='lower left')

plt.show()

In [ ]:
# Metallicity Distribution Functions (MDFs)
fig, axs = plt.subplots(1, 3, figsize=(12, 4))

names = ['W17_fid', 'W17_long_sfh', 'W17_short_sfh', 'W17_linexp']
colors = ['k', 'r', 'g', 'b']
labels = ['$\\tau_{sfh}=6.0$ Exp', '$\\tau_{sfh}=40.0$ Exp', '$\\tau_{sfh}=2.5$ Exp', '$\\tau_{sfh}=2.5$ Lin-Exp']
for i, name in enumerate(names):
    mdf = vice.mdf(name)
    bins = mdf['bin_edge_left'] + mdf['bin_edge_right'][-1:]
    axs[0].hist(mdf['bin_edge_left'], weights=mdf['dn/d[o/h]'], bins=bins, histtype='step', color=colors[i])
    axs[1].hist(mdf['bin_edge_left'], weights=mdf['dn/d[fe/h]'], bins=bins, histtype='step', color=colors[i])
    axs[2].hist(mdf['bin_edge_left'], weights=mdf['dn/d[fe/h]'], bins=bins, histtype='step', color=colors[i], label=labels[i])

ax = axs[0]
# ax.hist(mdf['bin_edge_left'], weights=mdf['dn/d[o/h]'], bins=bins, histtype='step', color='k')
ax.set_yscale('log')
ax.set_xlim((-1.6, 0.4))
ax.set_ylim((0.01, 20))
ax.xaxis.set_minor_locator(MultipleLocator(0.1))
ax.xaxis.set_major_locator(MultipleLocator(0.5))
ax.set_xlabel('[O/H]')
ax.set_ylabel('MDF in 0.05-dex bins')

ax = axs[1]
# ax.hist(mdf['bin_edge_left'], weights=mdf['dn/d[fe/h]'], bins=bins, histtype='step', color='k')
ax.set_yscale('log')
ax.set_xlim((-1.6, 0.4))
ax.set_ylim((0.01, 20))
ax.xaxis.set_minor_locator(MultipleLocator(0.1))
ax.xaxis.set_major_locator(MultipleLocator(0.5))
ax.set_xlabel('[Fe/H]')
ax.set_ylabel('MDF in 0.05-dex bins')

ax = axs[2]
# ax.hist(mdf['bin_edge_left'], weights=mdf['dn/d[fe/h]'], bins=bins, histtype='step', color='k')
ax.set_xlim((-1.6, 0.4))
ax.set_ylim((0, 10))
ax.xaxis.set_minor_locator(MultipleLocator(0.1))
ax.xaxis.set_major_locator(MultipleLocator(0.5))
ax.yaxis.set_minor_locator(MultipleLocator(0.5))
ax.yaxis.set_major_locator(MultipleLocator(2))
ax.set_xlabel('[Fe/H]')
ax.set_ylabel('MDF in 0.05-dex bins')
ax.legend(loc='upper left')

plt.show()

## Experimenting with multi-zone migration

In [12]:
mw = vice.milkyway(name='milkyway')
print(mw)

VICE supplementary data required, downloading now.
You will not need to repeat this process.
vice.milkyway{
    name -----------> milkyway
    n_zones --------> 40
    n_stars --------> 1
    verbose --------> False
    simple ---------> False
    annuli ---------> [0, 0.5, 1, ... , 19, 19.5, 20]
    evolution ------> <function milkyway.default_evolution at 0x7fdbbf3995e0>
    mode -----------> ifr
    elements -------> ('fe', 'sr', 'o')
    IMF ------------> kroupa
    mass_loading ---> <function milkyway.default_mass_loading at 0x7fdbbf399af0>
    dt -------------> 0.01
    bins -----------> [-3, -2.95, -2.9, ... , 0.9, 0.95, 1]
    delay ----------> 0.15
    RIa ------------> plaw
    smoothing ------> 0.0
    tau_ia ---------> 1.5
    m_upper --------> 100.0
    m_lower --------> 0.08
    postMS ---------> 0.1
    Z_solar --------> 0.014
}


In [15]:
tau_sfh = 6

def exp_sfr(t):
    """Exponentially declining star formation rate."""
    sfr = np.exp(-t/tau_sfh)
    return sfr

mw.func = exp_sfr
mw.mode = 'sfr'
mw.elements = ('fe', 'o')
mw.IMF = 'kroupa'
mw.eta = 2.5 # mass-loading parameter, or ratio of outflow rate to star formation rate
mw.recycling = 0.4 # recycling parameter, fraction of stellar mass from CCSNe and AGB stars returned at original metallicity
mw.delay = 0.15 # delay time of onset of SNe Ia [Gyr]
mw.RIa = 'exp' # SN Ia DTD
mw.tau_ia = 1.5 # exponential SN Ia timescale [Gyr]
mw.tau_star = 1 # SFE timescale, ratio of current gas mass to star formation rate [Gyr]

out = mw.run(np.linspace(0, 12.5, 1251), overwrite=True)
print(out)

/home/dubay.11/.local/lib/python3.8/site-packages/vice/core/pickles.py:305: UserWarning: Could not pickle function. The following attribute will not be saved with this output: stars
  warnings.warn("""\


FileNotFoundError: [Errno 2] No such file or directory: 'milkyway.vice/milkyway.vice/zone0.vice/yields/ccsne'

AttributeError: 'milkyway' object has no attribute 'out'